# Laten Factor Model

In [1]:
import pandas as pd
pd.options.display.max_columns = 1000
r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('./data/u.data', sep='\t', names=r_cols, usecols=range(3), encoding="ISO-8859-1")

m_cols = ['movie_id', 'title']
movies = pd.read_csv('./data/u.item', sep='|', names=m_cols, usecols=range(2), encoding="ISO-8859-1")

ratings = pd.merge(movies, ratings)

ratings.head()

,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


In [2]:
ratings[ratings.user_id == 0]

,movie_id,title,user_id,rating
6402,50,Star Wars (1977),0,5
18852,133,Gone with the Wind (1939),0,1
23859,172,"Empire Strikes Back, The (1980)",0,5


In [3]:
MoviePerUser = ratings['user_id'].value_counts()
print(MoviePerUser)
UserPerMovie = ratings['title'].value_counts()
print(UserPerMovie)

405    737
655    685
13     636
450    540
276    518
416    493
537    490
303    484
234    480
393    448
181    435
279    434
429    414
846    405
7      403
94     400
682    399
308    397
293    388
92     388
222    387
201    386
59     382
435    379
378    375
880    368
417    365
896    362
592    360
796    358
      ... 
636     20
572     20
571     20
441     20
888     20
824     20
631     20
309     20
242     20
558     20
685     20
812     20
364     20
300     20
873     20
809     20
166     20
740     20
36      20
866     20
418     20
34      20
926     20
93      20
732     20
475     20
596     20
147     20
19      20
0        3
Name: user_id, Length: 944, dtype: int64
Star Wars (1977)                                   584
Contact (1997)                                     509
Fargo (1996)                                       508
Return of the Jedi (1983)                          507
Liar Liar (1997)                                   485
English Patie

In [4]:
print(len(ratings))
# data = ratings[ratings['user_id'].isin(MoviePerUser[MoviePerUser > 20].index)] #only select users who rated more than 20 movies
# print(len(data))
data = ratings[ratings['title'].isin(UserPerMovie[UserPerMovie >= 100].index)] #only select movies which were rated more than 99 times
print(len(data))

100003
65011


In [5]:
len(MoviePerUser[MoviePerUser > 20].index)

911

In [6]:
data.head()

,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


In [7]:
print(data['user_id'].nunique()) #total number of users
print(data['title'].nunique()) #total number of movies

944
338


In [8]:
data[data['user_id'] == 0]

,movie_id,title,user_id,rating
6402,50,Star Wars (1977),0,5
18852,133,Gone with the Wind (1939),0,1
23859,172,"Empire Strikes Back, The (1980)",0,5


In [9]:
ratings.groupby('title').user_id.count().reset_index().sort_values(by = 'user_id',ascending = False).head() #top five movies

,title,user_id
1398,Star Wars (1977),584
333,Contact (1997),509
498,Fargo (1996),508
1234,Return of the Jedi (1983),507
860,Liar Liar (1997),485


In [10]:
from scipy.sparse import coo_matrix
data['user_id'] = data['user_id'].astype('category')
data['title'] = data['title'].astype('category')

/anaconda3/envs/FTDS/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/FTDS/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
len(data['title'].value_counts())

338

In [12]:
display(data['user_id'].head())
display(data['user_id'].cat.codes.head())
display(data['title'].head())
display(data['title'].cat.codes.head())

0    308
1    287
2    148
3    280
4     66
Name: user_id, dtype: category
Categories (944, int64): [0, 1, 2, 3, ..., 940, 941, 942, 943]

0    308
1    287
2    148
3    280
4     66
dtype: int16

0    Toy Story (1995)
1    Toy Story (1995)
2    Toy Story (1995)
3    Toy Story (1995)
4    Toy Story (1995)
Name: title, dtype: category
Categories (338, object): [101 Dalmatians (1996), 12 Angry Men (1957), 2001: A Space Odyssey (1968), Absolute Power (1997), ..., Wizard of Oz, The (1939), Wrong Trousers, The (1993), Young Frankenstein (1974), Young Guns (1988)]

0    313
1    313
2    313
3    313
4    313
dtype: int16

In [13]:
R = coo_matrix((data['rating'].astype('float64'), (data['user_id'].cat.codes, data['title'].cat.codes)))

In [14]:
print(R.shape)
display((R.data == 0).all())

(944, 338)


False

In [15]:
display(R.data[0])
data['rating'].head(10)

4.0

0    4
1    5
2    4
3    4
4    3
5    4
6    4
7    3
8    5
9    3
Name: rating, dtype: int64

In [16]:
print(R.row[0])
print(R.col[0])
print(R.data[(R.row == 308) & (R.col == 848)])
print(R.data[0])

308
313
[]
4.0


In [17]:
dict(enumerate(data['title'].cat.categories))

{0: '101 Dalmatians (1996)',
 1: '12 Angry Men (1957)',
 2: '2001: A Space Odyssey (1968)',
 3: 'Absolute Power (1997)',
 4: 'Abyss, The (1989)',
 5: 'Ace Ventura: Pet Detective (1994)',
 6: 'Adventures of Priscilla, Queen of the Desert, The (1994)',
 7: 'African Queen, The (1951)',
 8: 'Air Force One (1997)',
 9: 'Aladdin (1992)',
 10: 'Alien (1979)',
 11: 'Alien 3 (1992)',
 12: 'Alien: Resurrection (1997)',
 13: 'Aliens (1986)',
 14: 'Amadeus (1984)',
 15: 'American President, The (1995)',
 16: 'Amistad (1997)',
 17: 'Annie Hall (1977)',
 18: 'Apocalypse Now (1979)',
 19: 'Apollo 13 (1995)',
 20: 'Apt Pupil (1998)',
 21: 'Army of Darkness (1993)',
 22: 'Arsenic and Old Lace (1944)',
 23: 'As Good As It Gets (1997)',
 24: 'Austin Powers: International Man of Mystery (1997)',
 25: 'Babe (1995)',
 26: 'Back to the Future (1985)',
 27: 'Basic Instinct (1992)',
 28: 'Batman (1989)',
 29: 'Batman Forever (1995)',
 30: 'Batman Returns (1992)',
 31: 'Beautiful Girls (1996)',
 32: 'Beauty and

In [18]:
R.data

array([4., 5., 4., ..., 3., 2., 2.])

In [19]:
print(R)

  (308, 313)	4.0
  (287, 313)	5.0
  (148, 313)	4.0
  (280, 313)	4.0
  (66, 313)	3.0
  (5, 313)	4.0
  (109, 313)	4.0
  (181, 313)	3.0
  (95, 313)	5.0
  (268, 313)	3.0
  (189, 313)	5.0
  (145, 313)	3.0
  (158, 313)	4.0
  (67, 313)	3.0
  (232, 313)	4.0
  (150, 313)	4.0
  (289, 313)	3.0
  (117, 313)	4.0
  (49, 313)	2.0
  (223, 313)	4.0
  (56, 313)	4.0
  (17, 313)	4.0
  (340, 313)	5.0
  (177, 313)	3.0
  (194, 313)	4.0
  :	:
  (314, 209)	4.0
  (682, 209)	3.0
  (860, 209)	2.0
  (865, 209)	1.0
  (938, 209)	3.0
  (854, 209)	1.0
  (879, 209)	2.0
  (343, 209)	1.0
  (901, 209)	3.0
  (880, 209)	3.0
  (758, 209)	3.0
  (887, 209)	5.0
  (907, 209)	5.0
  (548, 209)	4.0
  (717, 209)	4.0
  (67, 209)	3.0
  (927, 209)	4.0
  (99, 209)	4.0
  (595, 209)	2.0
  (790, 209)	3.0
  (291, 209)	2.0
  (665, 209)	1.0
  (294, 209)	3.0
  (943, 209)	2.0
  (869, 209)	2.0


# Mean Squared Error

In [20]:
from numpy.linalg import norm
import numpy as np
#R = actual table from coo matrix, P = P table, Q = Q table, ld = learning
def meansquarederror(R, P, Q, ld):
    rating = R.data
    rows = R.row
    cols = R.col
    err = 0
    for ui in range(len(rating)):
        rui = rating[ui] #actual rating
        u = rows[ui]
        i = cols[ui]
        if rui > 0:
            err += pow(rui - np.dot(P[u,:], Q[:, i]),2) + ld * (norm(P[u,:])**2 + norm(Q[:,i])**2)
    return err
        
        
    

In [21]:
R.data < 0

array([False, False, False, ..., False, False, False])

# Gradient Descent

In [22]:
#R = actual R table, f = features, ld = lambda, lr = learning rate, epochs = steps
def SGD(R, f, ld = 0.02, lr = 0.001, epochs = 20):
    m, n = R.shape
    P = np.random.rand(m, f) #initialize values
    Q = np.random.rand(f, n) #initialize values
    rmse = np.sqrt(meansquarederror(R, P, Q, ld)/len(R.data))
    print(f'Initial RMSE: {rmse}')
    
    for epoch in range(epochs):
        for ui in range(len(R.data)):
            rui = R.data[ui] #actual rating
            u = R.row[ui] 
            i = R.col[ui]
            if rui > 0:
                eui = rui - np.dot(P[u,:], Q[:,i]) #error between actual and calculated
                P[u,:] = P[u,:] + lr * 2 * (eui * Q[:,i] - ld * P[u,:]) #update P table
                Q[:,i] = Q[:,i] + lr * 2 * (eui * P[u,:] - ld * Q[:,i]) #update Q table
        rmse = np.sqrt(meansquarederror(R, P, Q, ld) / len(R.data))
        if rmse < 0.5: #benchmark
            break
    print(f'Final RMSE: {rmse}')
    return P, Q
        
    

In [23]:
(P,Q) = SGD(R, 3, ld = 0.02, lr = 0.001, epochs = 10)

Initial RMSE: 3.1819166586979795
Final RMSE: 0.9954403022422585


In [24]:
R_new = np.dot(P,Q)

In [25]:
R_new.shape

(944, 338)

In [26]:
print(R.shape)

(944, 338)


In [27]:
R.row

array([308, 287, 148, ..., 294, 943, 869], dtype=int32)

# Result

In [28]:
R_new_df = pd.DataFrame(R_new)
R_new_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337
0,1.729150,2.584545,2.308428,2.124675,2.171749,1.739083,2.215030,2.381042,2.212952,2.195911,2.354194,1.597056,1.959719,2.228219,2.494499,2.105638,2.405076,2.299129,2.440044,2.371904,2.504483,2.006051,2.354790,2.631627,1.948166,2.392001,2.255882,1.789196,2.177878,1.549027,1.633139,1.943724,2.271002,1.811236,2.211215,2.235865,2.042339,2.297956,1.954081,2.242839,2.387379,2.256251,2.250090,2.305998,2.479752,2.247089,1.863565,2.489925,2.110150,2.394268,1.687145,2.394689,1.488835,1.961741,2.073543,2.581943,2.312920,2.425466,2.092454,2.513125,2.607392,2.158862,2.162881,2.253030,2.624912,2.067888,2.241281,2.011807,1.749594,2.131577,2.255787,2.316182,2.051976,2.031392,1.874583,1.658105,2.135697,2.020313,2.168463,2.226335,1.775660,2.108304,2.210337,2.283636,2.203197,2.231701,1.849296,2.235353,1.989160,1.897749,2.195953,1.528133,2.492439,1.738563,1.991200,2.282817,2.037094,2.257942,2.284623,2.510451,2.270715,1.805143,1.439575,2.043572,1.746208,2.001674,2.162496,2.306419,2.429120,1.764599,2.262219,2.015369,1.835458,1.763757,2.189031,2.256751,2.221198,2.198260,2.259711,1.980130,2.408036,2.180941,2.340347,2.018700,2.276944,2.441012,2.218921,1.655399,2.114049,2.100907,1.942703,2.407564,2.561422,2.552703,1.950510,2.208039,2.514612,2.221338,2.385679,2.456270,1.871132,2.432285,2.107535,2.203579,1.761841,2.000511,2.254044,2.185002,2.198428,2.523720,2.116449,1.720353,2.409022,2.021073,1.980222,2.338168,1.866586,2.215914,2.041494,2.181426,2.210860,2.041665,2.326974,1.749473,2.360431,2.242502,1.920494,1.883046,2.124733,2.177856,1.419316,2.218252,2.420267,1.908398,2.055285,2.417687,2.550171,2.081599,2.414408,2.136286,1.955757,2.317618,2.169795,2.178668,2.450018,2.029395,1.886633,1.751997,2.279828,2.213422,2.463465,2.597852,1.632047,2.235270,1.807638,1.900884,2.201770,1.949245,2.048064,1.670369,2.268290,1.865740,2.352736,2.259474,1.844519,2.175795,2.410509,1.974004,2.347746,1.559386,1.846213,2.220347,2.004159,2.202302,2.323632,1.949241,1.701694,1.815545,2.136179,1.799312,2.348432,2.601935,1.693938,2.142531,1.836302,2.489000,1.851668,2.345369,2.035899,2.172801,2.047141,2.239608,2.503596,2.128819,2.053953,2.104494,2.148347,2.302565,1.949983,2.140813,2.460790,2.011673,2.206708,2.418843,2.388293,2.078937,2.431570,2.532528,2.295997,2.203479,2.121227,2.070798,2.622014,2.316101,2.246782,2.305123,2.280639,1.971255,2.123113,2.379021,2.282925,2.104184,2.001752,1.845848,2.654588,2.075836,1.867270,2.310335,2.486328,2.415325,2.280538,2.026529,1.961468,2.665090,2.220734,2.282370,2.570445,2.332217,2.009153,2.105846,2.447239,2.029716,2.194745,2.365618,2.257181,1.733762,2.059231,2.187140,1.829491,1.920642,1.752711,2.119280,1.969564,1.782324,2.182400,2.531134,1.936167,1.955409,2.432075,2.239190,2.225090,2.438368,2.380519,2.258765,1.987182,2.171584,2

In [29]:
dict(enumerate(data['title'].cat.categories))

{0: '101 Dalmatians (1996)',
 1: '12 Angry Men (1957)',
 2: '2001: A Space Odyssey (1968)',
 3: 'Absolute Power (1997)',
 4: 'Abyss, The (1989)',
 5: 'Ace Ventura: Pet Detective (1994)',
 6: 'Adventures of Priscilla, Queen of the Desert, The (1994)',
 7: 'African Queen, The (1951)',
 8: 'Air Force One (1997)',
 9: 'Aladdin (1992)',
 10: 'Alien (1979)',
 11: 'Alien 3 (1992)',
 12: 'Alien: Resurrection (1997)',
 13: 'Aliens (1986)',
 14: 'Amadeus (1984)',
 15: 'American President, The (1995)',
 16: 'Amistad (1997)',
 17: 'Annie Hall (1977)',
 18: 'Apocalypse Now (1979)',
 19: 'Apollo 13 (1995)',
 20: 'Apt Pupil (1998)',
 21: 'Army of Darkness (1993)',
 22: 'Arsenic and Old Lace (1944)',
 23: 'As Good As It Gets (1997)',
 24: 'Austin Powers: International Man of Mystery (1997)',
 25: 'Babe (1995)',
 26: 'Back to the Future (1985)',
 27: 'Basic Instinct (1992)',
 28: 'Batman (1989)',
 29: 'Batman Forever (1995)',
 30: 'Batman Returns (1992)',
 31: 'Beautiful Girls (1996)',
 32: 'Beauty and

In [30]:
R_new_df.columns = data['title'].cat.categories
R_new_df.index = data['user_id'].cat.categories

In [31]:
R_new_df.head()

,101 Dalmatians (1996),12 Angry Men (1957),2001: A Space Odyssey (1968),Absolute Power (1997),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),"Adventures of Priscilla, Queen of the Desert, The (1994)","African Queen, The (1951)",Air Force One (1997),Aladdin (1992),Alien (1979),Alien 3 (1992),Alien: Resurrection (1997),Aliens (1986),Amadeus (1984),"American President, The (1995)",Amistad (1997),Annie Hall (1977),Apocalypse Now (1979),Apollo 13 (1995),Apt Pupil (1998),Army of Darkness (1993),Arsenic and Old Lace (1944),As Good As It Gets (1997),Austin Powers: International Man of Mystery (1997),Babe (1995),Back to the Future (1985),Basic Instinct (1992),Batman (1989),Batman Forever (1995),Batman Returns (1992),Beautiful Girls (1996),Beauty and the Beast (1991),Beavis and Butt-head Do America (1996),Being There (1979),Ben-Hur (1959),Benny & Joon (1993),Big Night (1996),"Birdcage, The (1996)","Birds, The (1963)",Blade Runner (1982),"Blues Brothers, The (1980)",Bonnie and Clyde (1967),Boogie Nights (1997),"Boot, Das (1981)",Bound (1996),Bram Stoker's Dracula (1992),Braveheart (1995),Brazil (1985),"Bridge on the River Kwai, The (1957)",Broken Arrow (1996),Butch Cassidy and the Sundance Kid (1969),"Cable Guy, The (1996)",Cape Fear (1991),Carrie (1976),Casablanca (1942),Chasing Amy (1997),Chinatown (1974),Cinderella (1950),Cinema Paradiso (1988),Citizen Kane (1941),Clear and Present Danger (1994),Clerks (1994),"Clockwork Orange, A (1971)","Close Shave, A (1995)",Clueless (1995),Cold Comfort Farm (1995),Con Air (1997),Conan the Barbarian (1981),Conspiracy Theory (1997),Contact (1997),Cool Hand Luke (1967),Cop Land (1997),Courage Under Fire (1996),"Craft, The (1996)",Crash (1996),Crimson Tide (1995),"Crow, The (1994)","Crying Game, The (1992)",Dances with Wolves (1990),Dante's Peak (1997),Dave (1993),Dead Man Walking (1995),Dead Poets Society (1989),"Deer Hunter, The (1978)","Devil's Advocate, The (1997)","Devil's Own, The (1997)",Die Hard (1988),Die Hard 2 (1990),Die Hard: With a Vengeance (1995),Donnie Brasco (1997),Down Periscope (1996),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963),Dragonheart (1996),Dumbo (1941),E.T. the Extra-Terrestrial (1982),Ed Wood (1994),"Edge, The (1997)",Emma (1996),"Empire Strikes Back, The (1980)","English Patient, The (1996)",Eraser (1996),Event Horizon (1997),Everyone Says I Love You (1996),Evita (1996),Executive Decision (1996),Face/Off (1997),Fantasia (1940),Fargo (1996),Father of the Bride Part II (1995),Field of Dreams (1989),"Fifth Element, The (1997)","Firm, The (1993)","First Wives Club, The (1996)","Fish Called Wanda, A (1988)",Fly Away Home (1996),Forrest Gump (1994),Four Weddings and a Funeral (1994),Fried Green Tomatoes (1991),"Frighteners, The (1996)","Fugitive, The (1993)",Full Metal Jacket (1987),"Full Monty, The (1997)",G.I. Jane (1997),"Game, The (1997)",Gandhi (1982),Gattaca (1997),George of the Jungle (1997),Get Shorty (1995),Ghost (1990),"Ghost and the Darkness, The (1996)",Glory (1989),"Godfather, The (1972)","Godfather: Part II, The (1974)",GoldenEye (1995),Gone with the Wind (1939),Good Will Hunting (1997),"Good, The Bad and The Ugly, The (1966)",GoodFellas (1990),"Graduate, The (1967)",Grease (1978),"Great Escape, The (1963)",Grosse Pointe Blank (1997),Groundhog Day (1993),Grumpier Old Men (1995),Happy Gilmore (1996),Harold and Maude (1971),Heat (1995),Heathers (1989),Henry V (1989),Highlander (1986),Home Alone (1990),Hoop Dreams (1994),"Hudsucker Proxy, The (1994)","Hunchback of Notre Dame, The (1996)","Hunt for Red October, The (1990)",I Know What You Did Last Summer (1997),"Ice Storm, The (1997)",In & Out (1997),In the Line of Fire (1993),In the Name of the Father (1993),Independence Day (ID4) (1996),Indiana Jones and the Last Crusade (1989),Interview with the Vampire (1994),It's a Wonderful Life (1946),Jackie Brown (1997),Jackie Chan's First Strike (1996),James and the Giant Peach (1996),Jaws (1975),Jerry Maguire (1996),Jungle2Jungle (1997),J

In [32]:
R_actual = pd.pivot_table(data = data, values = 'rating', index = data['user_id'].cat.codes, columns=data['title'].cat.codes)

In [33]:
R_actual.columns = data['title'].cat.categories

In [34]:
R_actual.index = data['user_id'].cat.categories

In [35]:
# data['user_id'].cat.codes

In [36]:
R_actual.head(10)

,101 Dalmatians (1996),12 Angry Men (1957),2001: A Space Odyssey (1968),Absolute Power (1997),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),"Adventures of Priscilla, Queen of the Desert, The (1994)","African Queen, The (1951)",Air Force One (1997),Aladdin (1992),Alien (1979),Alien 3 (1992),Alien: Resurrection (1997),Aliens (1986),Amadeus (1984),"American President, The (1995)",Amistad (1997),Annie Hall (1977),Apocalypse Now (1979),Apollo 13 (1995),Apt Pupil (1998),Army of Darkness (1993),Arsenic and Old Lace (1944),As Good As It Gets (1997),Austin Powers: International Man of Mystery (1997),Babe (1995),Back to the Future (1985),Basic Instinct (1992),Batman (1989),Batman Forever (1995),Batman Returns (1992),Beautiful Girls (1996),Beauty and the Beast (1991),Beavis and Butt-head Do America (1996),Being There (1979),Ben-Hur (1959),Benny & Joon (1993),Big Night (1996),"Birdcage, The (1996)","Birds, The (1963)",Blade Runner (1982),"Blues Brothers, The (1980)",Bonnie and Clyde (1967),Boogie Nights (1997),"Boot, Das (1981)",Bound (1996),Bram Stoker's Dracula (1992),Braveheart (1995),Brazil (1985),"Bridge on the River Kwai, The (1957)",Broken Arrow (1996),Butch Cassidy and the Sundance Kid (1969),"Cable Guy, The (1996)",Cape Fear (1991),Carrie (1976),Casablanca (1942),Chasing Amy (1997),Chinatown (1974),Cinderella (1950),Cinema Paradiso (1988),Citizen Kane (1941),Clear and Present Danger (1994),Clerks (1994),"Clockwork Orange, A (1971)","Close Shave, A (1995)",Clueless (1995),Cold Comfort Farm (1995),Con Air (1997),Conan the Barbarian (1981),Conspiracy Theory (1997),Contact (1997),Cool Hand Luke (1967),Cop Land (1997),Courage Under Fire (1996),"Craft, The (1996)",Crash (1996),Crimson Tide (1995),"Crow, The (1994)","Crying Game, The (1992)",Dances with Wolves (1990),Dante's Peak (1997),Dave (1993),Dead Man Walking (1995),Dead Poets Society (1989),"Deer Hunter, The (1978)","Devil's Advocate, The (1997)","Devil's Own, The (1997)",Die Hard (1988),Die Hard 2 (1990),Die Hard: With a Vengeance (1995),Donnie Brasco (1997),Down Periscope (1996),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963),Dragonheart (1996),Dumbo (1941),E.T. the Extra-Terrestrial (1982),Ed Wood (1994),"Edge, The (1997)",Emma (1996),"Empire Strikes Back, The (1980)","English Patient, The (1996)",Eraser (1996),Event Horizon (1997),Everyone Says I Love You (1996),Evita (1996),Executive Decision (1996),Face/Off (1997),Fantasia (1940),Fargo (1996),Father of the Bride Part II (1995),Field of Dreams (1989),"Fifth Element, The (1997)","Firm, The (1993)","First Wives Club, The (1996)","Fish Called Wanda, A (1988)",Fly Away Home (1996),Forrest Gump (1994),Four Weddings and a Funeral (1994),Fried Green Tomatoes (1991),"Frighteners, The (1996)","Fugitive, The (1993)",Full Metal Jacket (1987),"Full Monty, The (1997)",G.I. Jane (1997),"Game, The (1997)",Gandhi (1982),Gattaca (1997),George of the Jungle (1997),Get Shorty (1995),Ghost (1990),"Ghost and the Darkness, The (1996)",Glory (1989),"Godfather, The (1972)","Godfather: Part II, The (1974)",GoldenEye (1995),Gone with the Wind (1939),Good Will Hunting (1997),"Good, The Bad and The Ugly, The (1966)",GoodFellas (1990),"Graduate, The (1967)",Grease (1978),"Great Escape, The (1963)",Grosse Pointe Blank (1997),Groundhog Day (1993),Grumpier Old Men (1995),Happy Gilmore (1996),Harold and Maude (1971),Heat (1995),Heathers (1989),Henry V (1989),Highlander (1986),Home Alone (1990),Hoop Dreams (1994),"Hudsucker Proxy, The (1994)","Hunchback of Notre Dame, The (1996)","Hunt for Red October, The (1990)",I Know What You Did Last Summer (1997),"Ice Storm, The (1997)",In & Out (1997),In the Line of Fire (1993),In the Name of the Father (1993),Independence Day (ID4) (1996),Indiana Jones and the Last Crusade (1989),Interview with the Vampire (1994),It's a Wonderful Life (1946),Jackie Brown (1997),Jackie Chan's First Strike (1996),James and the Giant Peach (1996),Jaws (1975),Jerry Maguire (1996),Jungle2Jungle (1997),J

In [37]:
R_actual.loc[1].value_counts().sum()

173

In [38]:
# R_new_df.loc[1,R_actual.loc[1][R_actual.loc[1].isna()].index].sort_values(ascending = False)

In [66]:
pd.options.display.max_colwidth = 1000
def recommendation (user_id, R_calculated, R_actual, top_n = 5):
    assert type(user_id) == list,'Please enter a list!'
    assert top_n > 0, 'Please enter a number greater than 0'
    top_n = int(round(top_n,0))
    msg = len(f'|Top {int(top_n)} Recommendation|')
    print (' ' * 46 +'-' * (msg-2))
    print(' ' * 45 + f'|Top {top_n} Recommendation|')
    print (' ' * 46 +'-' * (msg-2))
    for i in user_id:
        print(f'-----------\nUser: {i}\n-----------')
        result = R_calculated.loc[i,R_actual.loc[i][R_actual.loc[i].isna()].index].sort_values(ascending = False).head(top_n)
        result = pd.DataFrame(result)
        result.columns = ['Rating']
        print (result, '\n')


In [69]:
recommendation([0,3,100,455],R_new_df, R_actual, top_n= 5)

                                              --------------------
                                             |Top 5 Recommendation|
                                              --------------------
-----------
User: 0
-----------
                                    Rating
Shawshank Redemption, The (1994)  2.665090
Schindler's List (1993)           2.654588
Titanic (1997)                    2.632067
As Good As It Gets (1997)         2.631627
Close Shave, A (1995)             2.624912 

-----------
User: 3
-----------
                                    Rating
Shawshank Redemption, The (1994)  3.594533
Close Shave, A (1995)             3.580981
Titanic (1997)                    3.574379
Rear Window (1954)                3.554269
As Good As It Gets (1997)         3.554155 

-----------
User: 100
-----------
                                    Rating
Close Shave, A (1995)             4.251006
Casablanca (1942)                 4.194769
Shawshank Redemption, The (1994)  4.115321
Schindle

# Use Collaborative Filtering with K nearest neighbor to recommend movies

In [ ]:
# data_remaining = ratings[ratings['user_id'].isin(MoviePerUser[MoviePerUser <= 20].index)]

In [41]:
whole_data =pd.pivot_table(data = data, columns = ['title'], index = ['user_id'], values = 'rating')
whole_data.shape

(944, 338)

In [42]:
# whole_data = R_.round() #round to 0 decimal places
whole_data.head()

title,101 Dalmatians (1996),12 Angry Men (1957),2001: A Space Odyssey (1968),Absolute Power (1997),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),"Adventures of Priscilla, Queen of the Desert, The (1994)","African Queen, The (1951)",Air Force One (1997),Aladdin (1992),Alien (1979),Alien 3 (1992),Alien: Resurrection (1997),Aliens (1986),Amadeus (1984),"American President, The (1995)",Amistad (1997),Annie Hall (1977),Apocalypse Now (1979),Apollo 13 (1995),Apt Pupil (1998),Army of Darkness (1993),Arsenic and Old Lace (1944),As Good As It Gets (1997),Austin Powers: International Man of Mystery (1997),Babe (1995),Back to the Future (1985),Basic Instinct (1992),Batman (1989),Batman Forever (1995),Batman Returns (1992),Beautiful Girls (1996),Beauty and the Beast (1991),Beavis and Butt-head Do America (1996),Being There (1979),Ben-Hur (1959),Benny & Joon (1993),Big Night (1996),"Birdcage, The (1996)","Birds, The (1963)",Blade Runner (1982),"Blues Brothers, The (1980)",Bonnie and Clyde (1967),Boogie Nights (1997),"Boot, Das (1981)",Bound (1996),Bram Stoker's Dracula (1992),Braveheart (1995),Brazil (1985),"Bridge on the River Kwai, The (1957)",Broken Arrow (1996),Butch Cassidy and the Sundance Kid (1969),"Cable Guy, The (1996)",Cape Fear (1991),Carrie (1976),Casablanca (1942),Chasing Amy (1997),Chinatown (1974),Cinderella (1950),Cinema Paradiso (1988),Citizen Kane (1941),Clear and Present Danger (1994),Clerks (1994),"Clockwork Orange, A (1971)","Close Shave, A (1995)",Clueless (1995),Cold Comfort Farm (1995),Con Air (1997),Conan the Barbarian (1981),Conspiracy Theory (1997),Contact (1997),Cool Hand Luke (1967),Cop Land (1997),Courage Under Fire (1996),"Craft, The (1996)",Crash (1996),Crimson Tide (1995),"Crow, The (1994)","Crying Game, The (1992)",Dances with Wolves (1990),Dante's Peak (1997),Dave (1993),Dead Man Walking (1995),Dead Poets Society (1989),"Deer Hunter, The (1978)","Devil's Advocate, The (1997)","Devil's Own, The (1997)",Die Hard (1988),Die Hard 2 (1990),Die Hard: With a Vengeance (1995),Donnie Brasco (1997),Down Periscope (1996),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963),Dragonheart (1996),Dumbo (1941),E.T. the Extra-Terrestrial (1982),Ed Wood (1994),"Edge, The (1997)",Emma (1996),"Empire Strikes Back, The (1980)","English Patient, The (1996)",Eraser (1996),Event Horizon (1997),Everyone Says I Love You (1996),Evita (1996),Executive Decision (1996),Face/Off (1997),Fantasia (1940),Fargo (1996),Father of the Bride Part II (1995),Field of Dreams (1989),"Fifth Element, The (1997)","Firm, The (1993)","First Wives Club, The (1996)","Fish Called Wanda, A (1988)",Fly Away Home (1996),Forrest Gump (1994),Four Weddings and a Funeral (1994),Fried Green Tomatoes (1991),"Frighteners, The (1996)","Fugitive, The (1993)",Full Metal Jacket (1987),"Full Monty, The (1997)",G.I. Jane (1997),"Game, The (1997)",Gandhi (1982),Gattaca (1997),George of the Jungle (1997),Get Shorty (1995),Ghost (1990),"Ghost and the Darkness, The (1996)",Glory (1989),"Godfather, The (1972)","Godfather: Part II, The (1974)",GoldenEye (1995),Gone with the Wind (1939),Good Will Hunting (1997),"Good, The Bad and The Ugly, The (1966)",GoodFellas (1990),"Graduate, The (1967)",Grease (1978),"Great Escape, The (1963)",Grosse Pointe Blank (1997),Groundhog Day (1993),Grumpier Old Men (1995),Happy Gilmore (1996),Harold and Maude (1971),Heat (1995),Heathers (1989),Henry V (1989),Highlander (1986),Home Alone (1990),Hoop Dreams (1994),"Hudsucker Proxy, The (1994)","Hunchback of Notre Dame, The (1996)","Hunt for Red October, The (1990)",I Know What You Did Last Summer (1997),"Ice Storm, The (1997)",In & Out (1997),In the Line of Fire (1993),In the Name of the Father (1993),Independence Day (ID4) (1996),Indiana Jones and the Last Crusade (1989),Interview with the Vampire (1994),It's a Wonderful Life (1946),Jackie Brown (1997),Jackie Chan's First Strike (1996),James and the Giant Peach (1996),Jaws (1975),Jerry Maguire (1996),Jungle2Jungle (19

In [43]:
from scipy.spatial.distance import hamming

In [44]:
def KNearestNeighbours(target_user, K):
    ls = []
    all_user = pd.DataFrame(whole_data.index)
    
    for user in target_user:
        all_user = all_user[all_user['user_id'] != user]
        #calculate hamming distance 
        all_user['distance'] = all_user['user_id'].apply(lambda x: 
                                                         hamming(whole_data[whole_data.index == x].T[x], 
                                                                 whole_data[whole_data.index == user].T[user]))
        result = all_user.sort_values(by = 'distance', ascending = True)[:K]
        ls.append(result)
    return ls
        

In [45]:
result = KNearestNeighbours([0, 9], 10) #testing
result[0]

,user_id,distance
1,1,0.994083
347,347,0.994083
350,350,0.994083
352,352,0.994083
363,363,0.994083
369,369,0.994083
373,373,0.994083
389,389,0.994083
392,392,0.994083
393,393,0.994083


In [70]:
def topN(target_user, N = 3, K = 10):
    assert type(target_user) == list, "Please enter a list!"
    assert N > 0, "Please enter a numeber greater than 0!"
    assert K > 0, "Please enter a numeber greater than 0!"
    ls = []
    N = int(round(N,0))
    K = int(round(K,0))
    nearestuser = KNearestNeighbours(target_user, K)
    print(f'\n[Top {N} Recommendation within {K} nearest neighbours]\n')
    for i in range(len(nearestuser)):
        nearestrating = whole_data[whole_data.index.isin(nearestuser[i]['user_id'])]
        averagerating = nearestrating.apply(np.nanmean, axis = 0).dropna()
        alreadywatch = whole_data[whole_data.index == target_user[i]].T.dropna() #watched movies
        topN = averagerating[~averagerating.index.isin(list(alreadywatch.index))].sort_values(ascending = False)[:N]
        ls.append(topN)
        print(f'\nUser: {target_user[i]}\n---------')
#         print(topN)
        for n in range(N):
            print (topN.index[n])

    return ls

    

In [72]:
recommendation = topN([0], N = 5, K = 100)


[Top 5 Recommendation within 100 nearest neighbours]


User: 0
---------
Return of the Jedi (1983)
Raiders of the Lost Ark (1981)
Close Shave, A (1995)
Casablanca (1942)
Schindler's List (1993)


# Use Item-Based Collaborative Filtering to recommend movies

In [49]:
data_item_based = pd.pivot_table(data = data, columns = 'title', index = 'user_id', values = 'rating')
data_item_based.head()

title,101 Dalmatians (1996),12 Angry Men (1957),2001: A Space Odyssey (1968),Absolute Power (1997),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),"Adventures of Priscilla, Queen of the Desert, The (1994)","African Queen, The (1951)",Air Force One (1997),Aladdin (1992),Alien (1979),Alien 3 (1992),Alien: Resurrection (1997),Aliens (1986),Amadeus (1984),"American President, The (1995)",Amistad (1997),Annie Hall (1977),Apocalypse Now (1979),Apollo 13 (1995),Apt Pupil (1998),Army of Darkness (1993),Arsenic and Old Lace (1944),As Good As It Gets (1997),Austin Powers: International Man of Mystery (1997),Babe (1995),Back to the Future (1985),Basic Instinct (1992),Batman (1989),Batman Forever (1995),Batman Returns (1992),Beautiful Girls (1996),Beauty and the Beast (1991),Beavis and Butt-head Do America (1996),Being There (1979),Ben-Hur (1959),Benny & Joon (1993),Big Night (1996),"Birdcage, The (1996)","Birds, The (1963)",Blade Runner (1982),"Blues Brothers, The (1980)",Bonnie and Clyde (1967),Boogie Nights (1997),"Boot, Das (1981)",Bound (1996),Bram Stoker's Dracula (1992),Braveheart (1995),Brazil (1985),"Bridge on the River Kwai, The (1957)",Broken Arrow (1996),Butch Cassidy and the Sundance Kid (1969),"Cable Guy, The (1996)",Cape Fear (1991),Carrie (1976),Casablanca (1942),Chasing Amy (1997),Chinatown (1974),Cinderella (1950),Cinema Paradiso (1988),Citizen Kane (1941),Clear and Present Danger (1994),Clerks (1994),"Clockwork Orange, A (1971)","Close Shave, A (1995)",Clueless (1995),Cold Comfort Farm (1995),Con Air (1997),Conan the Barbarian (1981),Conspiracy Theory (1997),Contact (1997),Cool Hand Luke (1967),Cop Land (1997),Courage Under Fire (1996),"Craft, The (1996)",Crash (1996),Crimson Tide (1995),"Crow, The (1994)","Crying Game, The (1992)",Dances with Wolves (1990),Dante's Peak (1997),Dave (1993),Dead Man Walking (1995),Dead Poets Society (1989),"Deer Hunter, The (1978)","Devil's Advocate, The (1997)","Devil's Own, The (1997)",Die Hard (1988),Die Hard 2 (1990),Die Hard: With a Vengeance (1995),Donnie Brasco (1997),Down Periscope (1996),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963),Dragonheart (1996),Dumbo (1941),E.T. the Extra-Terrestrial (1982),Ed Wood (1994),"Edge, The (1997)",Emma (1996),"Empire Strikes Back, The (1980)","English Patient, The (1996)",Eraser (1996),Event Horizon (1997),Everyone Says I Love You (1996),Evita (1996),Executive Decision (1996),Face/Off (1997),Fantasia (1940),Fargo (1996),Father of the Bride Part II (1995),Field of Dreams (1989),"Fifth Element, The (1997)","Firm, The (1993)","First Wives Club, The (1996)","Fish Called Wanda, A (1988)",Fly Away Home (1996),Forrest Gump (1994),Four Weddings and a Funeral (1994),Fried Green Tomatoes (1991),"Frighteners, The (1996)","Fugitive, The (1993)",Full Metal Jacket (1987),"Full Monty, The (1997)",G.I. Jane (1997),"Game, The (1997)",Gandhi (1982),Gattaca (1997),George of the Jungle (1997),Get Shorty (1995),Ghost (1990),"Ghost and the Darkness, The (1996)",Glory (1989),"Godfather, The (1972)","Godfather: Part II, The (1974)",GoldenEye (1995),Gone with the Wind (1939),Good Will Hunting (1997),"Good, The Bad and The Ugly, The (1966)",GoodFellas (1990),"Graduate, The (1967)",Grease (1978),"Great Escape, The (1963)",Grosse Pointe Blank (1997),Groundhog Day (1993),Grumpier Old Men (1995),Happy Gilmore (1996),Harold and Maude (1971),Heat (1995),Heathers (1989),Henry V (1989),Highlander (1986),Home Alone (1990),Hoop Dreams (1994),"Hudsucker Proxy, The (1994)","Hunchback of Notre Dame, The (1996)","Hunt for Red October, The (1990)",I Know What You Did Last Summer (1997),"Ice Storm, The (1997)",In & Out (1997),In the Line of Fire (1993),In the Name of the Father (1993),Independence Day (ID4) (1996),Indiana Jones and the Last Crusade (1989),Interview with the Vampire (1994),It's a Wonderful Life (1946),Jackie Brown (1997),Jackie Chan's First Strike (1996),James and the Giant Peach (1996),Jaws (1975),Jerry Maguire (1996),Jungle2Jungle (19

In [50]:
correlation = data_item_based.corr(method='pearson', min_periods = 100) #find correlation between each movie which has been rated at least by 20 users

In [51]:
correlation.head()

title,101 Dalmatians (1996),12 Angry Men (1957),2001: A Space Odyssey (1968),Absolute Power (1997),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),"Adventures of Priscilla, Queen of the Desert, The (1994)","African Queen, The (1951)",Air Force One (1997),Aladdin (1992),Alien (1979),Alien 3 (1992),Alien: Resurrection (1997),Aliens (1986),Amadeus (1984),"American President, The (1995)",Amistad (1997),Annie Hall (1977),Apocalypse Now (1979),Apollo 13 (1995),Apt Pupil (1998),Army of Darkness (1993),Arsenic and Old Lace (1944),As Good As It Gets (1997),Austin Powers: International Man of Mystery (1997),Babe (1995),Back to the Future (1985),Basic Instinct (1992),Batman (1989),Batman Forever (1995),Batman Returns (1992),Beautiful Girls (1996),Beauty and the Beast (1991),Beavis and Butt-head Do America (1996),Being There (1979),Ben-Hur (1959),Benny & Joon (1993),Big Night (1996),"Birdcage, The (1996)","Birds, The (1963)",Blade Runner (1982),"Blues Brothers, The (1980)",Bonnie and Clyde (1967),Boogie Nights (1997),"Boot, Das (1981)",Bound (1996),Bram Stoker's Dracula (1992),Braveheart (1995),Brazil (1985),"Bridge on the River Kwai, The (1957)",Broken Arrow (1996),Butch Cassidy and the Sundance Kid (1969),"Cable Guy, The (1996)",Cape Fear (1991),Carrie (1976),Casablanca (1942),Chasing Amy (1997),Chinatown (1974),Cinderella (1950),Cinema Paradiso (1988),Citizen Kane (1941),Clear and Present Danger (1994),Clerks (1994),"Clockwork Orange, A (1971)","Close Shave, A (1995)",Clueless (1995),Cold Comfort Farm (1995),Con Air (1997),Conan the Barbarian (1981),Conspiracy Theory (1997),Contact (1997),Cool Hand Luke (1967),Cop Land (1997),Courage Under Fire (1996),"Craft, The (1996)",Crash (1996),Crimson Tide (1995),"Crow, The (1994)","Crying Game, The (1992)",Dances with Wolves (1990),Dante's Peak (1997),Dave (1993),Dead Man Walking (1995),Dead Poets Society (1989),"Deer Hunter, The (1978)","Devil's Advocate, The (1997)","Devil's Own, The (1997)",Die Hard (1988),Die Hard 2 (1990),Die Hard: With a Vengeance (1995),Donnie Brasco (1997),Down Periscope (1996),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963),Dragonheart (1996),Dumbo (1941),E.T. the Extra-Terrestrial (1982),Ed Wood (1994),"Edge, The (1997)",Emma (1996),"Empire Strikes Back, The (1980)","English Patient, The (1996)",Eraser (1996),Event Horizon (1997),Everyone Says I Love You (1996),Evita (1996),Executive Decision (1996),Face/Off (1997),Fantasia (1940),Fargo (1996),Father of the Bride Part II (1995),Field of Dreams (1989),"Fifth Element, The (1997)","Firm, The (1993)","First Wives Club, The (1996)","Fish Called Wanda, A (1988)",Fly Away Home (1996),Forrest Gump (1994),Four Weddings and a Funeral (1994),Fried Green Tomatoes (1991),"Frighteners, The (1996)","Fugitive, The (1993)",Full Metal Jacket (1987),"Full Monty, The (1997)",G.I. Jane (1997),"Game, The (1997)",Gandhi (1982),Gattaca (1997),George of the Jungle (1997),Get Shorty (1995),Ghost (1990),"Ghost and the Darkness, The (1996)",Glory (1989),"Godfather, The (1972)","Godfather: Part II, The (1974)",GoldenEye (1995),Gone with the Wind (1939),Good Will Hunting (1997),"Good, The Bad and The Ugly, The (1966)",GoodFellas (1990),"Graduate, The (1967)",Grease (1978),"Great Escape, The (1963)",Grosse Pointe Blank (1997),Groundhog Day (1993),Grumpier Old Men (1995),Happy Gilmore (1996),Harold and Maude (1971),Heat (1995),Heathers (1989),Henry V (1989),Highlander (1986),Home Alone (1990),Hoop Dreams (1994),"Hudsucker Proxy, The (1994)","Hunchback of Notre Dame, The (1996)","Hunt for Red October, The (1990)",I Know What You Did Last Summer (1997),"Ice Storm, The (1997)",In & Out (1997),In the Line of Fire (1993),In the Name of the Father (1993),Independence Day (ID4) (1996),Indiana Jones and the Last Crusade (1989),Interview with the Vampire (1994),It's a Wonderful Life (1946),Jackie Brown (1997),Jackie Chan's First Strike (1996),James and the Giant Peach (1996),Jaws (1975),Jerry Maguire (1996),Jungle2Jungle (19

In [52]:
def item_based(target_user, N = 5):
    assert type(target_user) == list, 'Please enter a list!'
    print(f'\n[Top {N} Recommendation]\n')
    for user in target_user:
        print(f'\nUser: {user}\n-------')
        movie_watched = data_item_based[data_item_based.index == user].T.dropna()[user]
#         print(f'Watched: {movie_watched}')
        all_ = pd.Series()
        for movie in range(len(movie_watched)):
            correspondingmovies = correlation.loc[movie_watched.index[movie]] * movie_watched[movie] 
            correspondingmovies.dropna(inplace = True)
            all_ = all_.append(correspondingmovies)
        all_.sort_values(ascending = False, inplace = True)
        all_ = all_.groupby(all_.index).sum().sort_values(ascending= False)
        result = all_[~all_.index.isin(movie_watched.index)][:N]
#         print(result)
        for movie in result.index:
            print(f'{movie}')
    

In [59]:
item_based([0])


[Top 5 Recommendation]


User: 0
-------
Return of the Jedi (1983)
Raiders of the Lost Ark (1981)
Indiana Jones and the Last Crusade (1989)
Bridge on the River Kwai, The (1957)
Back to the Future (1985)


# Conclusion

In [58]:
data[data.user_id == 0]

,movie_id,title,user_id,rating
6402,50,Star Wars (1977),0,5
18852,133,Gone with the Wind (1939),0,1
23859,172,"Empire Strikes Back, The (1980)",0,5


# Futher Action